# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import os
import sys
import numpy as np
import pandas as pd
from sqlalchemy import create_engine 
from IPython.display import display # Allows the use of display() for DataFrames
import pickle
import time

from workspace_utils import active_session

import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
display(df)
X = df['message']
y = df[df.columns[4:]]
display(X)
display(y)

display(list(df.columns[4:]))
category_names = list(df.columns[4:])
display(category_names)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
5                   Information about the National Palace-
6                           Storm at sacred heart of jesus
7        Please, we need tents and water. We are in Sil...
8          I would like to receive the messages, thank you
9        I am in Croix-des-Bouquets. We have health iss...
10       There's nothing to eat and water, we starving ...
11       I am in Petionville. I need more information r...
12       I am in Thomassin number 32, in the area named...
13       Let's do it together, need food in Delma 75, i...
14       More information on the 4636 number in order f...
15       A Comitee in Delmas 19, Rue ( street ) Janvier...
16       We need food and water in Klecin 12. We are dy.

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        #text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    #stop_words = stopwords.words("english")

    #clean_tokens = []
    #for tok in tokens:
    #    clean_tok = lemmatizer.lemmatize(tok).lower().strip()
    #    clean_tokens.append(clean_tok)

    #clean_tokens = [lemmatizer.lemmatize(word).lower().strip() for word in tokens if word not in stop_words]
    clean_tokens = [lemmatizer.lemmatize(word).lower().strip() for word in tokens]
    
    return clean_tokens



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def model_pipeline():

    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LogisticRegression()))
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
print(X.shape)
print(y.shape)


(26028,)
(26028, 35)


In [7]:
start = time.time()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
model = model_pipeline()
model.fit(X_train, y_train)

end = time.time()
print(end - start)

19.666999578475952


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def evaluate_model(model, X_test, y_test, category_names):
    
    y_pred = model.predict(X_test)
    
    print(classification_report(y_test, y_pred, target_names=category_names))

        
evaluate_model(model, X_test, y_test, category_names)        


                        precision    recall  f1-score   support

               related       0.84      0.96      0.89      3959
               request       0.82      0.59      0.69       902
                 offer       0.00      0.00      0.00        25
           aid_related       0.76      0.68      0.72      2156
          medical_help       0.67      0.18      0.28       431
      medical_products       0.79      0.20      0.33       264
     search_and_rescue       0.92      0.08      0.15       151
              security       0.00      0.00      0.00       106
              military       0.73      0.11      0.19       175
                 water       0.78      0.51      0.62       344
                  food       0.82      0.63      0.71       586
               shelter       0.84      0.45      0.58       487
              clothing       0.76      0.20      0.32        79
                 money       0.81      0.10      0.18       131
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [25]:
def model_pipeline():

    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LogisticRegression()))
    ])
    
    parameters = {  
        'tfidf__smooth_idf': (True, False),
        'clf__estimator__penalty' : ['l1', 'l2']
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv



In [26]:
start = time.time()

with active_session():
    # do long-running work here
    model = model_pipeline()
    model.fit(X_train, y_train)
    
end = time.time()
print(end - start)

288.8482506275177


In [17]:
print("\nBest Parameters:", model.best_params_)


Best Parameters: {'clf__estimator__penalty': 'l1', 'tfidf__smooth_idf': True}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
def evaluate_model(model, X_test, y_test, category_names):
    
    y_pred = model.predict(X_test)
    
    metrics = []
    for i in range(len(category_names)):
        accuracy = category_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:, i])       
        metrics.append(accuracy)
    

    display(pd.DataFrame(metrics, columns=['Category', 'accuracy']).set_index('Category'))
    print(classification_report(y_test, y_pred, target_names=category_names))

        
evaluate_model(model, X_test, y_test, category_names)        

,accuracy
Category,
related,0.823473
request,0.905302
offer,0.995198
aid_related,0.774683
medical_help,0.928160
medical_products,0.957549
search_and_rescue,0.973876
security,0.979447
military,0.971187


                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      3959
               request       0.80      0.60      0.69       902
                 offer       0.00      0.00      0.00        25
           aid_related       0.76      0.67      0.71      2156
          medical_help       0.66      0.27      0.38       431
      medical_products       0.69      0.30      0.41       264
     search_and_rescue       0.74      0.15      0.25       151
              security       0.00      0.00      0.00       106
              military       0.67      0.29      0.40       175
                 water       0.76      0.60      0.67       344
                  food       0.81      0.74      0.77       586
               shelter       0.80      0.57      0.67       487
              clothing       0.76      0.41      0.53        79
                 money       0.67      0.31      0.42       131
        missing_people       0.61      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [33]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


def model_pipeline():

    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(LogisticRegression()))

    ])
    
    parameters = {  
        'features__text_pipeline__tfidf__smooth_idf': (True, False),
        'clf__estimator__penalty' : ['l1', 'l2']
    }    

    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

In [34]:
start = time.time()

with active_session():
    # do long-running work here
    model = model_pipeline()
    model.fit(X_train, y_train)

end = time.time()
print(end - start)

1251.0577764511108


In [35]:
def evaluate_model(model, X_test, y_test, category_names):
    
    y_pred = model.predict(X_test)
    
    metrics = []
    for i in range(len(category_names)):
        accuracy = category_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:, i])       
        metrics.append(accuracy)
    

    display(pd.DataFrame(metrics, columns=['Category', 'accuracy']).set_index('Category'))
    print(classification_report(y_test, y_pred, target_names=category_names))

        
evaluate_model(model, X_test, y_test, category_names)     

,accuracy
Category,
related,0.823473
request,0.905878
offer,0.995198
aid_related,0.775259
medical_help,0.928160
medical_products,0.957549
search_and_rescue,0.973876
security,0.979447
military,0.970995


                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      3959
               request       0.80      0.60      0.69       902
                 offer       0.00      0.00      0.00        25
           aid_related       0.76      0.67      0.71      2156
          medical_help       0.66      0.27      0.38       431
      medical_products       0.69      0.30      0.41       264
     search_and_rescue       0.74      0.15      0.25       151
              security       0.00      0.00      0.00       106
              military       0.66      0.29      0.40       175
                 water       0.76      0.60      0.67       344
                  food       0.81      0.74      0.77       586
               shelter       0.81      0.57      0.67       487
              clothing       0.75      0.38      0.50        79
                 money       0.67      0.31      0.42       131
        missing_people       0.61      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [36]:
model_filepath = 'classifier.pkl'

In [37]:
pickle.dump(model, open(model_filepath, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.